## Clasificación de documentos:

# Calculo del género de una película a raíz de su sinopsis

## Parte I: Introducción

Se requiere la implementación de un algoritmo que, previamente entrenado, calcule el __género principal__ de una película en base al conocimiento adquirido.

Los géneros que tendrá en cuenta el algoritmo son: ___acción___, ___comedia___, ___terror___, ___bélico___ y __western__.

Los __archivos que contienen la sinopsis__ (o equivalente, pudiendo ser también un breve resumen de la primera parte de la película) de las películas estarán distribuídos de la siguiente forma: 1) Si forman parte del conjunto de pruebas estarán dentro de la carpeta del conjunto de prueba sin más, que será la carpeta en la que el algoritmo, una vez entrenado, buscará sinopsis para categorizarlas 2) Si forman parte del conjunto de entrenamiento estarán dentro de la carpeta del conjunto de entrenamiento y __a su vez__ dentro de una carpeta que indique su género.

Inicialmente se planteó utilizar _html_ como __formato__ para almacenar los archivos con los que vamos a trabajar pero, dado que no ha sido posible encontrar una fuente común para extraer todas las sinopsis, finalmente se almacenarán como __texto plano__, trabajando de esta manera con archivos _.txt_.

__Los conocimientos requeridos por parte del usuario__ que ejecutará el algoritmo __son mínimos__: tan sólo necesita __colocar__ los textos, en el formato adecuado, en la carpeta indicada y __ejecutar__ el algoritmo en sí. Únicamente se requiere mayor interacción por parte del usuario si desea cambiar el funcionamiento del algorimo en sí, o ajustar el mismo, como por ejemplo si desea cambiar las palabras clave para cada una de las categorías.

## Parte II: Escaneo de archivos

Empezamos símplemente indicando qué __carpeta__ contiene el __conjunto de entrenamiento__ y el de __prueba__, además de las __categorías__, que no serán útiles más adelante.

In [1132]:
ruta_conjunto_entrenamiento = "conjunto_entrenamiento"
ruta_conjunto_prueba = "conjunto_prueba"

Empezamos escaneando la carpeta del conjunto de entrenamiento, que contendrá las __categorías__ en las que se podrán clasificar los nuevos documentos:

In [1133]:
import os # Nos ayudaremos de la librería "os" para leer ficheros y carpetas.

categorías = {elemento for elemento in os.listdir(ruta_conjunto_entrenamiento) if os.path.isdir(ruta_conjunto_entrenamiento + "/" + elemento)} # "os.listdir" devuelve las el contenido de un directorio dado, pero además queremos filtrar que sea un directorio, por eso lo procesamos y le aplicamos el filtro de que sea un directorio.

print(categorías)

{'terror', 'western', 'comedia', 'acción', 'bélico'}


A continuación, vamos a recorrer nuestra estructura de carpetas para detectar cada uno de los textos a analizar.

Procedemos a __encontrar__ todos los __archivos ya clasificados__ (conjunto de entrenamiento), según su __categoría__.

In [1134]:
# NOTA: Vamos a almacenarlos en conjunto (set) ya que no nos interesa el orden y, además, no permite duplicados (no puede haber dos archivos con el mismo nombre).

archivos_entrenamiento = set()

archivos_entrenamiento_acción = set()
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "acción"):
    if file.endswith(".txt"):
        aux = "acción" + "/" + file
        archivos_entrenamiento.add(aux)
        archivos_entrenamiento_acción.add(aux)

archivos_entrenamiento_comedia = set()
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "comedia"):
    if file.endswith(".txt"):
        aux = "comedia" + "/" + file
        archivos_entrenamiento.add(aux)
        archivos_entrenamiento_comedia.add(aux)

archivos_entrenamiento_terror = set()
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "terror"):
    if file.endswith(".txt"):
        aux = "terror" + "/" + file
        archivos_entrenamiento.add(aux)
        archivos_entrenamiento_terror.add(aux)

archivos_entrenamiento_bélico = set()
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "bélico"):
    if file.endswith(".txt"):
        aux = "bélico" + "/" + file
        archivos_entrenamiento.add(aux)
        archivos_entrenamiento_bélico.add(aux)

archivos_entrenamiento_western = set()
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "western"):
    if file.endswith(".txt"):
        aux = "western" + "/" + file
        archivos_entrenamiento.add(aux)
        archivos_entrenamiento_western.add(aux)

Y, a continuación, procedemos a __encontrar__ los __archivos que querríamos clasificar__ (conjunto de prueba):

In [1135]:
# Usamos conjunto por la misma razón de arriba.

archivos_prueba = set()
for file in os.listdir(ruta_conjunto_prueba):
    if file.endswith(".txt"):
        archivos_prueba.add(file)

print(archivos_prueba)

{'Le llamaban Trinidad.txt', 'Misión imposible - Protocolo fantasma.txt', 'Shrek 3.txt', 'Teléfono rojo - Volamos hacia Moscú.txt', 'Sin perdón.txt', 'El padrino.txt', 'El jinete pálido.txt', 'El tirador.txt', 'Jungla de cristal.txt', 'A todo gas - Tokyo race.txt', 'La caza del Octubre Rojo.txt', 'Django desencadenado.txt', 'El ultimátum de Bourne.txt', 'Solo ante el peligro.txt', 'Fast and Furious 5.txt'}


## Parte III: Elección del conjunto de palabras clave

Para ayudarnos en el estudio de las __palabras clave__ que debemos escoger para cada categoría vamos a realizar un pequeño estudio para determinar las palabras más frecuentes de cada categoría. La elección en sí debemos realizarla a mano puesto que no podemos escoger directamente las más frecuentes puesto que con total seguridad entre las más frecuentes se encontrarán verbos, conectores, preposiciones, pronombres, artículos, nombres propios, etc. que no nos serán de utilidad a la hora de determinar la categoría de una película.

Debemos definir __método que recibirá__ tanto __un conjunto de archivos__ como una __ruta__ donde se encuentran y __contará las palabras__ que aparecen en él __y el número de veces que dichas palabras aparecen__.

Pero antes, definimos un método que reciba un solo archivo y cuente sus palabras. Este metodo será usado por el método que estamos buscando y los dividimos de esta forma porque debemos buscar en un solo archivo cuando apliquemos los algoritmos de Naive Bayes y kNN.

In [1136]:
def cuenta_palabras_desde_archivo(ruta, archivo):
    cuenta_palabras = {}
    
    file = open(ruta + "/" + archivo, "r", encoding="latin-1") # Elegimos latin-1 en vez de utf-8 por problemas con las tildes.

    for palabra in file.read().split(): # Recorremos el fichero, palabra a palabra.
        palabra = palabra.lower() # Pasamos la palabra a minúscula.
        # Para perder la menor información posible, reemplazamos ':', ',', ':' y ';', que son los carácteres más típicos que nos podemos encontrar adyacentes a una palabra y que la invalidarían en el siguiente if del algoritmo.
        palabra = palabra.replace('.', '')
        palabra = palabra.replace(',', '')
        palabra = palabra.replace(':', '')
        palabra = palabra.replace(';', '')
        if palabra.isalpha() is True: # Será true cuando todos los caracteres son alfabéticos y hay al menos uno.
            if palabra in cuenta_palabras:
                cuenta_palabras[palabra] += 1 # Si la palabra ya existe, entonces incrementa en 1 el número de veces que hace aparición.
            else:
                cuenta_palabras[palabra] = 1 # Si la palabra no existe, la añade (con valor 1 al número de veces que aparece).

    cuenta_palabras
    return cuenta_palabras

Ahora sí, definimos el método que estamos buscando y que se anunciaba antes:

In [1137]:
from collections import Counter # Lo usaremos para añadir un diccionario a otro.

def cuenta_palabras_desde_archivos(ruta, archivos):
    cuenta_palabras = {}
    
    for archivo in archivos:
        new = cuenta_palabras_desde_archivo(ruta, archivo)
        cuenta_palabras = dict(Counter(cuenta_palabras)+Counter(new))
    
    return cuenta_palabras

Además, con propósito de limpiar el output que obtendremos vamos a establecer un __umbral__ para desechar todas las palabras que se repitan por debajo del mismo:

In [1138]:
umbral_repetición = 4

Y, con el mismo propósito, otro __umbral__ para desechar todas las palabras con una longitud menor a él:

In [1139]:
umbral_longitud = 4

Y, por último, un método que use al anterior que, además, nos __ordene las palabras__ (de mayor a menor uso) __y las muestre__:

In [1140]:
def cuenta_palabras_desde_archivos_ordenadas(ruta, archivos):
    cuenta_palabras = cuenta_palabras_desde_archivos(ruta, archivos)
    
    lista = [] # Usamos una lista para poder ordenar las palabras.
    for palabra, contador in cuenta_palabras.items():
        lista.append((contador, palabra))

    lista = sorted(lista, reverse = True) # Lo ordenamos y lo invertimos para que las palabras más frecuentes estén arriba.

    for aux in lista:
        if(aux[0] >= umbral_repetición and len(aux[1]) >= umbral_longitud): # Si la palabra supera los umbrales indicados, se muestra.
            print("%s \t %d" % (aux[1], aux[0]))

Palabras más frecuentes en: __ACCIÓN__.

In [1141]:
cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_acción)

para 	 54
brian 	 29
después 	 24
bryan 	 24
pero 	 22
policía 	 20
está 	 20
mientras 	 19
ethan 	 19
coche 	 17
quien 	 16
mcclane 	 16
letty 	 16
donde 	 16
dice 	 15
tiene 	 14
ella 	 14
equipo 	 13
como 	 13
martin 	 12
cuando 	 12
agente 	 12
novia 	 11
hombres 	 11
entonces 	 11
encuentra 	 11
ellos 	 11
cual 	 11
hombre 	 10
dominic 	 10
tras 	 9
llega 	 9
hunt 	 9
aeropuerto 	 9
toretto 	 8
sobre 	 8
owen 	 8
luego 	 8
hija 	 8
había 	 8
auto 	 8
ahora 	 8
virus 	 7
todo 	 7
tanto 	 7
también 	 7
shaw 	 7
plan 	 7
nueva 	 7
mills 	 7
lenore 	 7
gasolina 	 7
están 	 7
durante 	 7
desde 	 7
comienza 	 7
antes 	 7
ambrosio 	 7
vive 	 6
unidos 	 6
todos 	 6
tego 	 6
sindicato 	 6
reúne 	 6
puede 	 6
película 	 6
parte 	 6
nyah 	 6
lugar 	 6
lleva 	 6
leonore 	 6
john 	 6
informa 	 6
hasta 	 6
grupo 	 6
fuera 	 6
estados 	 6
entrega 	 6
embargo 	 6
director 	 6
descubre 	 6
cuerpo 	 6
ciudad 	 6
casa 	 6
carrera 	 6
biocyte 	 6
bill 	 6
banda 	 6
ayuda 	 6
amigo 	 6
ángeles 	 5
vid

Palabras más frecuentes en: __COMEDIA__.

In [1142]:
cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_comedia)

para 	 47
como 	 25
esta 	 17
cuando 	 16
vida 	 15
humor 	 15
comedia 	 14
película 	 13
tiene 	 12
también 	 12
todo 	 11
sobre 	 11
pero 	 11
gags 	 11
está 	 11
mundo 	 10
hilarantes 	 10
entre 	 10
después 	 10
situaciones 	 9
país 	 9
hasta 	 9
años 	 9
todos 	 8
tiempo 	 8
serie 	 8
nueva 	 8
noche 	 8
mike 	 8
ellos 	 8
divertida 	 8
amigos 	 8
personajes 	 7
personaje 	 7
historia 	 7
donde 	 7
casa 	 7
borat 	 7
aunque 	 7
york 	 6
trabajo 	 6
sátira 	 6
risas 	 6
momentos 	 6
irreverente 	 6
hacen 	 6
fiona 	 6
embargo 	 6
divertido 	 6
dictador 	 6
desde 	 6
absurdo 	 6
unos 	 5
unidos 	 5
tras 	 5
toda 	 5
tienen 	 5
tener 	 5
shrek 	 5
quien 	 5
problemas 	 5
poco 	 5
pasar 	 5
otros 	 5
muchos 	 5
mucho 	 5
monstruos 	 5
mientras 	 5
kevin 	 5
incluso 	 5
hacer 	 5
guerra 	 5
gran 	 5
gloria 	 5
fuerte 	 5
este 	 5
ejército 	 5
durante 	 5
decide 	 5
caballeros 	 5
bien 	 5
baron 	 5
algunos 	 5
tópicos 	 4
sulley 	 4
suerte 	 4
solo 	 4
simpáticos 	 4
siempre 	 4
sido 	

Palabras más frecuentes en: __TERROR__.

In [1143]:
cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_terror)

lorraine 	 17
casa 	 17
carolyn 	 17
aunque 	 15
cuando 	 12
april 	 11
tras 	 10
ella 	 10
alguien 	 10
todo 	 8
mujer 	 8
donde 	 8
sobre 	 7
siendo 	 7
para 	 7
habitación 	 7
exorcismo 	 7
espíritu 	 7
warren 	 6
roger 	 6
muñeca 	 6
cómo 	 6
armario 	 6
viendo 	 5
ruidos 	 5
observando 	 5
noche 	 5
mientras 	 5
judy 	 5
hija 	 5
hacia 	 5
está 	 5
como 	 5
allí 	 5
todos 	 4
tiene 	 4
sótano 	 4
suelo 	 4
siguiente 	 4
rory 	 4
puerta 	 4
padre 	 4
lugar 	 4
hijo 	 4
hijas 	 4
familia 	 4
escucha 	 4
entretanto 	 4
encontrando 	 4
dice 	 4
descubre 	 4
debe 	 4
cual 	 4
contra 	 4
consigue 	 4
christine 	 4
annabelle 	 4


Palabras más frecuentes en: __BÉLICO__.

In [1144]:
cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_bélico)

para 	 44
guerra 	 39
como 	 24
ejército 	 22
sargento 	 21
soldados 	 19
está 	 18
durante 	 18
general 	 16
cuando 	 16
grupo 	 14
sobre 	 12
hombres 	 12
hitler 	 12
combate 	 12
este 	 11
tres 	 10
tras 	 10
todo 	 10
patoso 	 10
misión 	 10
mientras 	 10
después 	 10
ante 	 10
unidos 	 9
quien 	 9
pero 	 9
matar 	 9
estados 	 9
contra 	 9
teniente 	 8
stauffenberg 	 8
pelotón 	 8
james 	 8
ellos 	 8
capitán 	 8
alemán 	 8
alemanes 	 8
vietnam 	 7
tiene 	 7
tiempo 	 7
solo 	 7
recibe 	 7
raine 	 7
policías 	 7
parte 	 7
japonés 	 7
isla 	 7
historia 	 7
estadounidenses 	 7
estadounidense 	 7
entre 	 7
edificio 	 7
donde 	 7
coronel 	 7
compañeros 	 7
comienza 	 7
batalla 	 7
albert 	 7
vietnamita 	 6
todos 	 6
también 	 6
soldado 	 6
sido 	 6
regimiento 	 6
primera 	 6
pesar 	 6
otros 	 6
ordena 	 6
operación 	 6
oficial 	 6
nuevo 	 6
norman 	 6
marina 	 6
mando 	 6
kuribayashi 	 6
infantería 	 6
hombre 	 6
están 	 6
equipo 	 6
desde 	 6
bajo 	 6
algren 	 6
zaitsev 	 5
vida 	 5
tro

Palabras más frecuentes en: __WESTERN__.

In [1145]:
cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_western)

pueblo 	 32
para 	 31
pero 	 28
cuando 	 24
como 	 20
donde 	 15
hijo 	 14
joven 	 13
entre 	 13
amigo 	 13
tras 	 12
sheriff 	 12
pistolero 	 11
banda 	 11
historia 	 10
guerra 	 10
thorton 	 9
también 	 9
stoddard 	 9
película 	 9
mujer 	 9
mismo 	 9
hombre 	 9
familia 	 9
está 	 9
esposa 	 9
diligencia 	 9
camino 	 9
unos 	 8
quien 	 8
pequeño 	 8
parte 	 8
mientras 	 8
había 	 8
grupo 	 8
evans 	 8
encuentran 	 8
ante 	 8
allí 	 8
además 	 8
tiempo 	 7
rojo 	 7
padre 	 7
morgan 	 7
mccabe 	 7
llegan 	 7
llega 	 7
hombres 	 7
hermano 	 7
este 	 7
embargo 	 7
después 	 7
consigue 	 7
ciudad 	 7
belden 	 7
años 	 7
antes 	 7
vida 	 6
venganza 	 6
todos 	 6
tiene 	 6
shane 	 6
poco 	 6
otro 	 6
nombre 	 6
matt 	 6
hasta 	 6
hacia 	 6
ganado 	 6
fuerte 	 6
encuentra 	 6
ejército 	 6
dunson 	 6
decide 	 6
aunque 	 6
tres 	 5
trata 	 5
toda 	 5
tendero 	 5
sargento 	 5
regresa 	 5
recompensas 	 5
recompensa 	 5
recibe 	 5
pesar 	 5
méxico 	 5
muerte 	 5
manera 	 5
hermanos 	 5
finalmente 

Una vez realizado el estudio, inicializamos manualmente las __palabras clave__ de cada __categoría__, usaremos unas 20 palabras para cada una de ellas (podrán repetirse entre categorías).

In [1146]:
# Las inicializamos como conjuntos ya que no nos interesa el orden y, además, no se permiten duplicados.
palabras_clave_acción = {"policía", "coche", "agente", "auto", "escapar", "carrera", "seguridad", "llamada", "gasolina", "problemas", "operación", "escapa", "venganza", "prisión", "muerte", "hospital", "carreras", "ataque", "arma", "asesino", "asesina", "asesinos", "terrorista", "terroristas", "persecución", "muerto", "investigación", "destrucción", "bomba"}
palabras_clave_comedia = {"incorrecto", "simpáticos", "alegre", "carcajadas", "cómica", "cómicos", "delirante", "divertidos", "irreverente", "risas", "sátira", "divertido", "divertida", "gags", "hilarantes", "humor", "tópicos", "crítica", "absurdo", "hilarante", "absurdas", "extravagante"}
palabras_clave_terror = {}
palabras_clave_bélico = {"guerra", "ejército", "sargento", "soldados", "general", "grupo", "hombres", "combate", "misión", "unidos", "matar", "teniente", "capitán", "alemán", "alemanes", "vietnam", "estadounidense", "estadounidenses", "soldado", "regimiento", "operación", "oficial", "infantería", "cabo", "enemigo"}
palabras_clave_western = {"pueblo", "hijo", "joven", "amigo", "sheriff", "pistolero", "banda", "guerra", "diligencia", "camino", "duelo", "bandidos", "granjero", "asesino", "revólver", "arma", "armas", "desierto", "oeste"}

Y, a continuación, añadimos todas las palabras clave de cada categoría a un nuevo conjunto que contenga __todas las palabras clave__.

In [1147]:
palabras_clave = set()
# Update nos permite añadir el contenido de un set a otro set
palabras_clave.update(palabras_clave_acción)
palabras_clave.update(palabras_clave_comedia)
palabras_clave.update(palabras_clave_terror)
palabras_clave.update(palabras_clave_bélico)
palabras_clave.update(palabras_clave_western)
print("Las palabras clave son: %s" % (palabras_clave))

Las palabras clave son: {'carreras', 'muerto', 'divertida', 'hijo', 'sargento', 'grupo', 'sheriff', 'enemigo', 'hospital', 'bandidos', 'amigo', 'capitán', 'delirante', 'bomba', 'misión', 'infantería', 'escapar', 'regimiento', 'banda', 'revólver', 'cabo', 'risas', 'estadounidenses', 'muerte', 'pueblo', 'problemas', 'gasolina', 'escapa', 'sátira', 'terroristas', 'alegre', 'investigación', 'duelo', 'seguridad', 'agente', 'prisión', 'asesinos', 'joven', 'armas', 'vietnam', 'absurdo', 'pistolero', 'auto', 'ejército', 'simpáticos', 'humor', 'arma', 'absurdas', 'soldado', 'irreverente', 'destrucción', 'cómica', 'llamada', 'coche', 'asesina', 'carcajadas', 'divertido', 'venganza', 'soldados', 'cómicos', 'incorrecto', 'hilarante', 'general', 'carrera', 'combate', 'diligencia', 'crítica', 'divertidos', 'camino', 'estadounidense', 'unidos', 'granjero', 'matar', 'oficial', 'terrorista', 'hilarantes', 'alemán', 'guerra', 'tópicos', 'extravagante', 'operación', 'ataque', 'gags', 'policía', 'persecuc

## Parte IV: Procesamiento

En esta parte se va a llevar a cabo el procesamiento de datos para posteriormente utilizarlos en los algoritmos de __Naive Bayes__ y __kNN__.

Puesto que se deben realizar cálculos distintos para cada algoritmo, dividiremos esta sección en dos subsecciones:

### Parte IV-A: Procesamiento de Naive Bayes

Para aplicar el algoritmo Naive Bayes primero debemos calcular todos los __P(c)__ ___(probabilidad de "c")___ y los __P(t|c)__ ___(probabilidad de "t" condicionada a "c")___.

En este caso __"c"__ sería nuestra categoría y __"t"__ cada palabra clave.

Primero, vamos a calcular los __P(c)__. Para ello, tenemos que contar el número de documentos de la categoría en cuestión existentes en nuestro conjunto de entrenamiento y dividirlo entre el número total de documentos de nuestro conjunto de entrenamiento. Así pues, por ejemplo, la probabilidad de acción (__P(acción)__) sería el número resultante de dividir el total de documentos catalogados como "acción" de nuestro conjunto de entrenamiento entre el número total de documentos del conjunto de entrenamiento.

Como tenemos un conjunto con todos los archivos de entrenamiento y un conjunto específico por cada categoría:

In [1148]:
probabilidad_acción = len(archivos_entrenamiento_acción) / len(archivos_entrenamiento)
print("P(acción) = \t %f" % (probabilidad_acción))

P(acción) = 	 0.233333


In [1149]:
probabilidad_comedia = len(archivos_entrenamiento_comedia) / len(archivos_entrenamiento)
print("P(comedia) = \t %f" % (probabilidad_comedia))

P(comedia) = 	 0.266667


In [1150]:
probabilidad_terror = len(archivos_entrenamiento_terror) / len(archivos_entrenamiento)
print("P(terror) = \t %f" % (probabilidad_terror))

P(terror) = 	 0.011111


In [1151]:
probabilidad_bélico = len(archivos_entrenamiento_bélico) / len(archivos_entrenamiento)
print("P(bélico) = \t %f" % (probabilidad_bélico))

P(bélico) = 	 0.233333


In [1152]:
probabilidad_western = len(archivos_entrenamiento_western) / len(archivos_entrenamiento)
print("P(western) = \t %f" % (probabilidad_western))

P(western) = 	 0.255556


Sólo para asegurarnos, todas las probabilidades deben sumar __~1__ en este apartado:

In [1153]:
print("Suma de probabilidades (debe ser ~1) \t %f" % (probabilidad_acción + probabilidad_comedia + probabilidad_terror + probabilidad_bélico + probabilidad_western))

Suma de probabilidades (debe ser ~1) 	 1.000000


Ahora, para calcular los __P(t|c)__ será un poco más complejo. Para llevar a cabo esta tarea haremos uso de __un diccionario por cada categoría__ que __relacionará palabras clave con su probabilidad condicionada a la categoría del diccionario__. De nuevo, tomaremos la categoría "acción" como ejemplo: el diccionario "probabilidad_palabraclave_acción" recogerá, por cada palabra clave, su probabilidad condiccionada a la categoría acción, es decir, su __P(t|acción__), siendo "t" cada entrada del diccionario.

Pero antes de empezar, definiremos un método, que utilizaremos en los siguientes pasos, que genere cada diccionario deseado como salida y, además, le __aplique un suavizado de LaPlace__:

In [1154]:
def crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_categoría, palabras_clave_categoría):
    probabilidades_condicionadas = {}
    cuenta_palabras_categoría = cuenta_palabras_desde_archivos(ruta_conjunto_entrenamiento, archivos_entrenamiento_categoría) # Almacena las palabras clave de la categoría y el número de veces que se repiten.
    
    número_palabras_clave_totales = len(palabras_clave) # Número de palabras clave que poseemos en total.
    número_palabras_clave_categoría = sum(cuenta_palabras_categoría.values())
    
    for palabra_clave in palabras_clave:
        if palabra_clave in cuenta_palabras_categoría:
            número_veces_aparece_palabra_en_categoría = cuenta_palabras_categoría[palabra_clave] # Número de veces que la palabra clave se repite en esta categoría.
        else:
            número_veces_aparece_palabra_en_categoría = 0
        result = ((número_veces_aparece_palabra_en_categoría + 1) / (número_palabras_clave_categoría + número_palabras_clave_totales)) # Añadimos 1 en el numerador y el número de palabras claves totales en el denominador para aplicar el suavizado.
        probabilidades_condicionadas[palabra_clave] = result
    
    return probabilidades_condicionadas

También vamos a crear un pequeño método que nos ayude a visualizar las probabilidades condicionadas:

In [1155]:
def mostrar_diccionario_probabilidades_condicionadas(diccionario, categoría):
    for entrada in diccionario:
        print("P(%s|%s) = \t %f" % (entrada, categoría, diccionario[entrada]))

Ahora, empezamos con el cálculo en sí:

In [1156]:
probabilidad_palabraclave_acción = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_acción, palabras_clave_acción)

mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave_acción, "acción")

P(carreras|acción) = 	 0.000795
P(muerto|acción) = 	 0.000636
P(divertida|acción) = 	 0.000159
P(hijo|acción) = 	 0.000636
P(sargento|acción) = 	 0.000318
P(grupo|acción) = 	 0.001113
P(sheriff|acción) = 	 0.000159
P(enemigo|acción) = 	 0.000159
P(hospital|acción) = 	 0.000795
P(bandidos|acción) = 	 0.000159
P(amigo|acción) = 	 0.001113
P(capitán|acción) = 	 0.000318
P(delirante|acción) = 	 0.000159
P(bomba|acción) = 	 0.000636
P(misión|acción) = 	 0.000954
P(infantería|acción) = 	 0.000159
P(escapar|acción) = 	 0.000954
P(regimiento|acción) = 	 0.000159
P(banda|acción) = 	 0.001113
P(revólver|acción) = 	 0.000159
P(cabo|acción) = 	 0.000159
P(risas|acción) = 	 0.000159
P(estadounidenses|acción) = 	 0.000318
P(muerte|acción) = 	 0.000795
P(pueblo|acción) = 	 0.000159
P(problemas|acción) = 	 0.000477
P(gasolina|acción) = 	 0.001272
P(escapa|acción) = 	 0.000636
P(sátira|acción) = 	 0.000159
P(terroristas|acción) = 	 0.000318
P(alegre|acción) = 	 0.000159
P(investigación|acción) = 	 0.00

In [1157]:
probabilidad_palabraclave_comedia = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_comedia, palabras_clave_comedia)

mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave_comedia, "comedia")

P(carreras|comedia) = 	 0.000179
P(muerto|comedia) = 	 0.000179
P(divertida|comedia) = 	 0.001612
P(hijo|comedia) = 	 0.000537
P(sargento|comedia) = 	 0.000179
P(grupo|comedia) = 	 0.000896
P(sheriff|comedia) = 	 0.000358
P(enemigo|comedia) = 	 0.000179
P(hospital|comedia) = 	 0.000358
P(bandidos|comedia) = 	 0.000179
P(amigo|comedia) = 	 0.000717
P(capitán|comedia) = 	 0.000179
P(delirante|comedia) = 	 0.000896
P(bomba|comedia) = 	 0.000179
P(misión|comedia) = 	 0.000537
P(infantería|comedia) = 	 0.000179
P(escapar|comedia) = 	 0.000179
P(regimiento|comedia) = 	 0.000179
P(banda|comedia) = 	 0.000179
P(revólver|comedia) = 	 0.000179
P(cabo|comedia) = 	 0.000358
P(risas|comedia) = 	 0.001254
P(estadounidenses|comedia) = 	 0.000179
P(muerte|comedia) = 	 0.000358
P(pueblo|comedia) = 	 0.000717
P(problemas|comedia) = 	 0.001075
P(gasolina|comedia) = 	 0.000179
P(escapa|comedia) = 	 0.000358
P(sátira|comedia) = 	 0.001254
P(terroristas|comedia) = 	 0.000179
P(alegre|comedia) = 	 0.000717
P

In [1158]:
probabilidad_palabraclave_terror = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_terror, palabras_clave_terror)

mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave_terror, "terror")

P(carreras|terror) = 	 0.000501
P(muerto|terror) = 	 0.000501
P(divertida|terror) = 	 0.000501
P(hijo|terror) = 	 0.002503
P(sargento|terror) = 	 0.000501
P(grupo|terror) = 	 0.001001
P(sheriff|terror) = 	 0.001001
P(enemigo|terror) = 	 0.000501
P(hospital|terror) = 	 0.000501
P(bandidos|terror) = 	 0.000501
P(amigo|terror) = 	 0.000501
P(capitán|terror) = 	 0.000501
P(delirante|terror) = 	 0.000501
P(bomba|terror) = 	 0.000501
P(misión|terror) = 	 0.000501
P(infantería|terror) = 	 0.000501
P(escapar|terror) = 	 0.000501
P(regimiento|terror) = 	 0.000501
P(banda|terror) = 	 0.000501
P(revólver|terror) = 	 0.000501
P(cabo|terror) = 	 0.001001
P(risas|terror) = 	 0.000501
P(estadounidenses|terror) = 	 0.000501
P(muerte|terror) = 	 0.000501
P(pueblo|terror) = 	 0.001001
P(problemas|terror) = 	 0.000501
P(gasolina|terror) = 	 0.000501
P(escapa|terror) = 	 0.001001
P(sátira|terror) = 	 0.000501
P(terroristas|terror) = 	 0.000501
P(alegre|terror) = 	 0.000501
P(investigación|terror) = 	 0.00

In [1159]:
probabilidad_palabraclave_bélico = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_bélico, palabras_clave_bélico)

mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave_bélico, "bélico")

P(carreras|bélico) = 	 0.000162
P(muerto|bélico) = 	 0.000485
P(divertida|bélico) = 	 0.000162
P(hijo|bélico) = 	 0.000323
P(sargento|bélico) = 	 0.003554
P(grupo|bélico) = 	 0.002423
P(sheriff|bélico) = 	 0.000162
P(enemigo|bélico) = 	 0.000969
P(hospital|bélico) = 	 0.000162
P(bandidos|bélico) = 	 0.000162
P(amigo|bélico) = 	 0.000808
P(capitán|bélico) = 	 0.001454
P(delirante|bélico) = 	 0.000162
P(bomba|bélico) = 	 0.000323
P(misión|bélico) = 	 0.001777
P(infantería|bélico) = 	 0.001131
P(escapar|bélico) = 	 0.000646
P(regimiento|bélico) = 	 0.001131
P(banda|bélico) = 	 0.000162
P(revólver|bélico) = 	 0.000323
P(cabo|bélico) = 	 0.000969
P(risas|bélico) = 	 0.000162
P(estadounidenses|bélico) = 	 0.001292
P(muerte|bélico) = 	 0.000808
P(pueblo|bélico) = 	 0.000485
P(problemas|bélico) = 	 0.000162
P(gasolina|bélico) = 	 0.000162
P(escapa|bélico) = 	 0.000162
P(sátira|bélico) = 	 0.000162
P(terroristas|bélico) = 	 0.000162
P(alegre|bélico) = 	 0.000162
P(investigación|bélico) = 	 0.00

In [1160]:
probabilidad_palabraclave_western = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_western, palabras_clave_western)

mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave_western, "western")

P(carreras|western) = 	 0.000172
P(muerto|western) = 	 0.000687
P(divertida|western) = 	 0.000172
P(hijo|western) = 	 0.002576
P(sargento|western) = 	 0.001030
P(grupo|western) = 	 0.001545
P(sheriff|western) = 	 0.002232
P(enemigo|western) = 	 0.000172
P(hospital|western) = 	 0.000172
P(bandidos|western) = 	 0.001030
P(amigo|western) = 	 0.002404
P(capitán|western) = 	 0.001030
P(delirante|western) = 	 0.000343
P(bomba|western) = 	 0.000172
P(misión|western) = 	 0.000343
P(infantería|western) = 	 0.000172
P(escapar|western) = 	 0.000515
P(regimiento|western) = 	 0.000343
P(banda|western) = 	 0.002060
P(revólver|western) = 	 0.000515
P(cabo|western) = 	 0.000343
P(risas|western) = 	 0.000172
P(estadounidenses|western) = 	 0.000343
P(muerte|western) = 	 0.001030
P(pueblo|western) = 	 0.005666
P(problemas|western) = 	 0.000687
P(gasolina|western) = 	 0.000172
P(escapa|western) = 	 0.000172
P(sátira|western) = 	 0.000172
P(terroristas|western) = 	 0.000172
P(alegre|western) = 	 0.000172
P

Una vez calculadas todas las probabilidades condicionadas de todas las categorías ya hemos finalizado con este subapartado, pero antes vamos a recordar un par de cosas:

Para acceder a una probabilidad específica símplemente ejecutamos lo siguiente (para el ejemplo obtendremos la __probabilidad de coche__ condicionada a la categoría __acción__, es decir, __P(coche|acción)__):

In [1161]:
print("P(coche|acción) = %f" % (probabilidad_palabraclave_acción["coche"]))

P(coche|acción) = 0.002862


Si hubiese una __palabra clave que no pertenece a acción, la probabilidad no sería 0__ ya que estamos usando suavizado. Por ejemplo, podemos comprobarlo con la probabilidad de soldado (que no forma parte de las palabras clave de acción) condicionada a acción (__P(soldado|acción)__):

In [1162]:
print("P(soldado|acción) = %f" % (probabilidad_palabraclave_acción["soldado"]))

P(soldado|acción) = 0.000159


### Parte IV-B: Procesamiento de kNN

In [1163]:
palabras_clave_y_repeticiones = {}

cuenta_palabras = cuenta_palabras_desde_archivos(ruta_conjunto_entrenamiento, archivos_entrenamiento)

for palabra in cuenta_palabras:
    if palabra in palabras_clave:
        palabras_clave_y_repeticiones[palabra] = cuenta_palabras[palabra]

print(palabras_clave_y_repeticiones)

{'capitán': 14, 'guerra': 56, 'estadounidense': 10, 'ejército': 34, 'regimiento': 7, 'asesino': 11, 'joven': 19, 'armas': 8, 'grupo': 33, 'combate': 14, 'soldados': 25, 'general': 19, 'irreverente': 6, 'tópicos': 4, 'divertida': 8, 'unidos': 24, 'pueblo': 38, 'bandidos': 5, 'llamada': 8, 'banda': 17, 'duelo': 5, 'arma': 10, 'policía': 28, 'seguridad': 7, 'incorrecto': 4, 'gags': 12, 'absurdo': 6, 'sátira': 6, 'risas': 6, 'amigo': 26, 'extravagante': 3, 'humor': 16, 'divertido': 6, 'hilarantes': 10, 'hilarante': 4, 'cómica': 3, 'delirante': 5, 'cómicos': 3, 'escapa': 5, 'simpáticos': 4, 'alegre': 3, 'vietnam': 7, 'soldado': 10, 'sargento': 27, 'misión': 18, 'enemigo': 5, 'hijo': 24, 'infantería': 6, 'oficial': 11, 'cabo': 8, 'teniente': 12, 'hospital': 5, 'divertidos': 3, 'crítica': 4, 'estadounidenses': 9, 'hombres': 31, 'revólver': 3, 'camino': 12, 'problemas': 10, 'carcajadas': 3, 'agente': 12, 'venganza': 11, 'muerte': 14, 'escapar': 10, 'sheriff': 14, 'coche': 20, 'asesina': 5, 'pe

In [1164]:
def calculo_peso(palabra_clave, ruta, archivo):
    frecuencia_en_documento = 0
    frecuencia_documental = 0
    frecuencia_documental_inversa = 0
    peso = 0
    
    cuenta_palabras = cuenta_palabras_desde_archivo(ruta, archivo)
    if palabra_clave in cuenta_palabras:
        frecuencia_en_documento = cuenta_palabras[palabra_clave]
    
    frecuencia_documental = palabras_clave_y_repeticiones[palabra_clave]
    
    if frecuencia_documental is not 0:
        aux = len(archivos_entrenamiento) / frecuencia_documental
    else:
        aux = 0
    
    frecuencia_documental_inversa = math.log(aux)
    
    peso = frecuencia_en_documento * frecuencia_documental_inversa
    
    return peso

In [1165]:
def calculo_pesos(ruta, archivo):
    lista_pesos = []
    
    for palabra_clave in palabras_clave:
        lista_pesos.append(calculo_peso(palabra_clave, ruta, archivo))
    
    return lista_pesos

In [1166]:
import math

diccionario_palabraclave_pesos = {}

for archivo in archivos_entrenamiento:
    lista_pesos_archivo = calculo_pesos(ruta_conjunto_entrenamiento, archivo)
    
    diccionario_palabraclave_pesos[archivo] = lista_pesos_archivo

# print(diccionario_palabraclave_pesos)

## Parte V: Salvado del procesamiento en fichero

### Parte V-A: Salvado del procesamiento de Naive Bayes

Dado que el enunciado de la práctica requiere que guardemos el procesado que acabamos que realizar (en la parte IV) para después utilizarlo en la ejecución de los algoritmos, procedemos a ello.

Primero necesitamos crear dos grandes listas: la primera contendrá la cadena relacionada con la probabilidad de que pase algo y la segunda la probabilidad de que pase. Ambas listas deben ser iguales y debemos almacenar toda la información obtenida.

In [1167]:
lista_texto = []
lista_valor = []

lista_texto.append("P(acción)")
lista_valor.append(probabilidad_acción)

lista_texto.append("P(comedia)")
lista_valor.append(probabilidad_comedia)

lista_texto.append("P(terror)")
lista_valor.append(probabilidad_terror)

lista_texto.append("P(bélico)")
lista_valor.append(probabilidad_bélico)

lista_texto.append("P(western)")
lista_valor.append(probabilidad_western)

for entrada in probabilidad_palabraclave_acción:
    lista_texto.append("P(" + entrada + "|" + "acción" + ")")
    lista_valor.append(probabilidad_palabraclave_acción[entrada])

for entrada in probabilidad_palabraclave_comedia:
    lista_texto.append("P(" + entrada + "|" + "comedia" + ")")
    lista_valor.append(probabilidad_palabraclave_comedia[entrada])

for entrada in probabilidad_palabraclave_terror:
    lista_texto.append("P(" + entrada + "|" + "terror" + ")")
    lista_valor.append(probabilidad_palabraclave_terror[entrada])

for entrada in probabilidad_palabraclave_bélico:
    lista_texto.append("P(" + entrada + "|" + "bélico" + ")")
    lista_valor.append(probabilidad_palabraclave_bélico[entrada])

for entrada in probabilidad_palabraclave_western:
    lista_texto.append("P(" + entrada + "|" + "western" + ")")
    lista_valor.append(probabilidad_palabraclave_western[entrada])

In [1168]:
import csv # Librería que necesitaremos para guardar en formato ".csv".

csvfile = "csv/naive-bayes.csv"
datos_a_guardar = zip(lista_texto, lista_valor)

with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for dato in datos_a_guardar:
        writer.writerow([dato[0]] + [dato[1]])

### Parte V-B: Salvado del procesamiento de kNN

In [1169]:
csvfile = "csv/knn.csv"
datos_a_guardar = diccionario_palabraclave_pesos

with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for dato in datos_a_guardar:
        writer.writerow([dato] + [datos_a_guardar[dato]])

## Parte VI: Ejecución de los algoritmos

En la parte II ya encontramos los ficheros del conjunto de test, que están en su respectiva carpeta, ahora tenemos que procesaros igual que hicimos en la parte III con los ficheros del conjunto de pruebas.

Vamos a definir un método que nos será de utilidad: a partir de un __csv__ dado y de una __cadena a buscar__, devolverá el valor relacionado con dicha celda del csv. Esto nos será útil para, a partir de la cadena de una probabilidad (por ejemplo, _P(bala|acción)_), nos devuelva su probabilidad.

In [1170]:
def lee_fichero(nombre_csv):
    with open(nombre_csv, 'rt', encoding="latin-1") as fichero:
        lector = csv.reader(fichero)
        diccionario = dict(lector)
    
    return diccionario

### Parte VI-A: Ejecución de Naive Bayes

El método naive_bayes recibe un archivo y los datos procesados (probabilidades) como parámetros y determina la categoría del archivo pasado como parámetro.

In [1171]:
def naive_bayes(archivo, csv):
    cuenta_palabras = cuenta_palabras_desde_archivo(ruta_conjunto_prueba, archivo)
    
    palabras_coincidentes_con_palabras_clave = cuenta_palabras.copy()
    
    # De las palabras que contiene el fichero, desechamos todas las que no coinciden con las palabras clave.
    for palabra in cuenta_palabras:
        if palabra not in palabras_clave:
            del palabras_coincidentes_con_palabras_clave[palabra]
    
    # Abrimos el fichero ".csv" generado para consultar datos en el siguiente paso.
    datos = lee_fichero(csv)
    
    # Ejecutamos el algoritmo en sí.
    resultados = {} # Resultados será un diccionario que contendrá la categoría y la "puntuación" otorgada por el algoritmo a esa categoría (para posteriormente elegir la categoría con el máximo valor).
    
    for categoría in categorías:
        probabilidades_condicionadas_a_multiplicar = []
        for palabra_clave in palabras_coincidentes_con_palabras_clave:
            cadena_a_buscar = "P(" + palabra_clave + "|" + categoría + ")" # Define la cadena que se debe buscar en el archivo. En este caso la probabilidad condicionada a la categoría.
            probabilidades_condicionadas_a_multiplicar.append(float(datos[cadena_a_buscar]) ** palabras_coincidentes_con_palabras_clave[palabra_clave]) # Busca el valor de la probabilidad condicionada requerida (lo transforma en float), lo eleva al número de veces que se repite y añade el valor de la probabilidad condicionada hallada a una lista que se pasará a multiplicar después.
        
        cadena_a_buscar = "P(" + categoría + ")" # Define la cadena que se debe buscar en el archivo. En este caso la probabilidad de la categoría.
        probabilidad_categoría = float(datos[cadena_a_buscar])
        
        # Multiplicamos los elementos de la lista de probabilidades condicionadas
        probabilidades_condicionadas_multiplicadas = 1.0
        for elemento in probabilidades_condicionadas_a_multiplicar:
            probabilidades_condicionadas_multiplicadas = probabilidades_condicionadas_multiplicadas * elemento
        
        resultados[categoría] = probabilidades_condicionadas_multiplicadas * probabilidad_categoría # Calcula el coeficiente
    
    result = max(resultados, key=resultados.get) # Devuleve el resultado del algoritmo. En este caso el elemento del diccionario con mayor coeficiente.
    
    return result

__Ejecutamos el algoritmo__ e __imprimimos__ los __resultados__ obtenidos:

In [1172]:
def aplicar_naive_bayes_archivos_prueba():
    print("El resultado de aplicar el algoritmo [Naive-Bayes] al conjunto de pruebas es...")
    for archivo in archivos_prueba:
        algoritmo = naive_bayes(archivo, "csv/naive-bayes.csv")
        print("[%s] \t [%s]" % (archivo.replace(".txt", ""), algoritmo))

In [1173]:
aplicar_naive_bayes_archivos_prueba()

El resultado de aplicar el algoritmo [Naive-Bayes] al conjunto de pruebas es...
[Le llamaban Trinidad] 	 [western]
[Misión imposible - Protocolo fantasma] 	 [acción]
[Shrek 3] 	 [comedia]
[Teléfono rojo - Volamos hacia Moscú] 	 [bélico]
[Sin perdón] 	 [western]
[El padrino] 	 [western]
[El jinete pálido] 	 [western]
[El tirador] 	 [acción]
[Jungla de cristal] 	 [acción]
[A todo gas - Tokyo race] 	 [acción]
[La caza del Octubre Rojo] 	 [bélico]
[Django desencadenado] 	 [western]
[El ultimátum de Bourne] 	 [acción]
[Solo ante el peligro] 	 [western]
[Fast and Furious 5] 	 [acción]


### Parte VI-B: Ejecución de kNN

In [1174]:
def calcula_distancia(v, w):
    # Comprueba que las listas son del mismo tamaño.
    if(len(v) == len(w)):
        numerador = sum([elemento_v * elemento_w for elemento_v, elemento_w in zip(v,w)])
        
        denominador_parte_v = math.sqrt(sum([elemento_v ** 2 for elemento_v in v]))
        denominador_parte_w = math.sqrt(sum([elemento_w ** 2 for elemento_w in w]))
        
        denominador = denominador_parte_v * denominador_parte_w
        
        return numerador / denominador

In [1175]:
def knn(archivo, csv):
    v = calculo_pesos(ruta_conjunto_prueba, archivo)
    
    # Abrimos el fichero ".csv" generado para consultar datos en el siguiente paso.
    datos = lee_fichero(csv)
    
    # Ejecutamos el algoritmo en sí.
    resultados = {} # Resultados será un diccionario que contendrá el archivo y la "puntuación" (similitud) otorgada por el algoritmo a esa categoría (para posteriormente elegir la categoría del archivo con la similitud más cercana a uno, que será el mayor valor).
    
    # Ahora que tenemos el peso del archivo a clasificar mediante el algoritmo y los pesos de los archivos del conjunto de entrenamiento (extraídos del ".csv" y guardados en forma de diccionario) tenemos que calcular, una por una, la distancia a cada elemento del conjunto de entrenamiento y quedarnos con la menor.
    for dato in datos:
        # Como lo que guardamos es una cadena, es necesario un pequeño procesamiento para transformarlo de nuevo en una lista.
        w = datos[dato].replace('[', '') # Primero eliminamos "[".
        w = w.replace(']', '') # Hacemos lo mismo con "]".
        w = w.split(",") # Aplicamos ".split()" para volver a "trocear" la cadena y convertirla de nuevo en una lista.
        w = [float(elemento) for elemento in w]
        resultados[dato] = calcula_distancia(v, w) # "v" son los pesos del archivo a clasificar y "w" los del archivo del conjunto de entrenamiento que está siendo procesado.
    
    # Para una mayor exactitud, saber en qué categoría se enmarcará la muestra y a qué película se debe, usaremos una lista para devolver dicha información (el elemento 0 contendrá la categoría y el 1 la película de dónde procede).
    result = max(resultados, key=resultados.get)
    result = result.split("/")
    
    return result[0], result[1]

In [1176]:
def aplicar_knn_archivos_prueba():
    print("El resultado de aplicar el algoritmo [kNN] al conjunto de pruebas es...")
    for archivo in archivos_prueba:
        algoritmo = knn(archivo, "csv/knn.csv")
        print("[%s] \t [%s] (por similitud con [%s])" % (archivo.replace(".txt", ""), algoritmo[0], algoritmo[1].replace(".txt", "")))

In [1177]:
aplicar_knn_archivos_prueba()

El resultado de aplicar el algoritmo [kNN] al conjunto de pruebas es...
[Le llamaban Trinidad] 	 [western] (por similitud con [El dorado])
[Misión imposible - Protocolo fantasma] 	 [acción] (por similitud con [Misión imposible - Nación secreta])
[Shrek 3] 	 [comedia] (por similitud con [Swiss Army Man])
[Teléfono rojo - Volamos hacia Moscú] 	 [bélico] (por similitud con [Senderos de gloria])
[Sin perdón] 	 [western] (por similitud con [Duelo de titanes])
[El padrino] 	 [acción] (por similitud con [La jungla 4 punto 0])
[El jinete pálido] 	 [acción] (por similitud con [Kill Bill - Volumen 1])
[El tirador] 	 [acción] (por similitud con [Venganza])
[Jungla de cristal] 	 [acción] (por similitud con [Misión imposible - Nación secreta])
[A todo gas - Tokyo race] 	 [acción] (por similitud con [Turbo-Charged Prelude])
[La caza del Octubre Rojo] 	 [bélico] (por similitud con [Banderas de nuestros padres])
[Django desencadenado] 	 [western] (por similitud con [Incidente en Ox-Bow])
[El ultimátum

## Parte VII: Análisis de los resultados

## Parte VIII: Conclusiones